# Analisi dei Dati di Mercato Statunitense

In [1]:
# Importazione Librerie

import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

In questo progetto ho analizzato i dati storici di diverse azioni del mercato statunitense. I dataset sono stati ottenuti da [Yahoo Finance](https://finance.yahoo.com).

**Obiettivi dell'analisi**:

- Esaminare l'andamento storico degli indici principali

- Valutare la volatilità annuale degli indici

- Analizzare il volume medio annuale degli indici

- Confrontare la performance annuale dei principali settori

- Calcolare le correlazioni tra i rendimenti delle azioni FAANG

- Analizzare la correlazione tra rendimenti delle azioni e materie prime

- Eseguire un backtesting di una strategia di trading basata sulle medie mobili

---


### **Trend Storico degli Indici Principali (Ultimi 10 anni)**

- SPY (S&P 500)

- DIA (Dow Jones Industrial Average)

- QQQ (NASDAQ-100)

In [2]:
# Caricamento dei dati
spy = pd.read_csv('Stocks/SPY.csv')
dia = pd.read_csv('Stocks/DIA.csv')
qqq = pd.read_csv('Stocks/QQQ.csv')

# Formattazione Datetime
spy['Date'] = pd.to_datetime(spy['Date'])
dia['Date'] = pd.to_datetime(dia['Date'])
qqq['Date'] = pd.to_datetime(qqq['Date'])

# Selezione solo delle colonne 'Close' (prezzo di chiusura)
spy_close = spy[['Date', 'Close']].rename(columns={'Close': 'SPY'})
dia_close = dia[['Date', 'Close']].rename(columns={'Close': 'DIA'})
qqq_close = qqq[['Date', 'Close']].rename(columns={'Close': 'QQQ'})

# Unione dei dati in un unico DataFrame
df = pd.merge(spy_close, dia_close, on='Date', how='inner')
df = pd.merge(df, qqq_close, on='Date', how='inner')

# Creazione del grafico interattivo con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['SPY'], mode='lines', name='SPY'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['DIA'], mode='lines', name='DIA'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['QQQ'], mode='lines', name='QQQ'))

# Configurazione dell'hover
fig.update_layout(
    hovermode='x unified',  # Mostra le etichette per tutti e tre gli indici
    title='Trend Storico degli Indici Principali (Ultimi 10 anni)',
    xaxis_title='Data',
    yaxis_title='Prezzo di Chiusura'
)

# Formattazione dell'hover per mostrare il giorno specifico
fig.update_xaxes(
    hoverformat='%Y-%m-%d'  # Mostra il formato completo della data
)

# Mostra il grafico
fig.show()



**Osservazioni:**

- Tutti gli indici mostrano una resilienza post-2020.

- Il settore tecnologico (QQQ) ha avuto una crescita superiore rispetto agli altri indici.

### **Volatilità Annuale degli Indici Principali (2014-2023)**

- SPY (S&P 500)

- DIA (Dow Jones Industrial Average)

- QQQ (NASDAQ-100)


In [3]:
# Funzione per calcolare la volatilità annuale
def annual_volatility(df, symbol):
    df = df.copy()  # Copia del DataFrame per evitare SettingWithCopyWarning
    df['Daily Return'] = df['Close'].pct_change()
    df = df.dropna()
    df['Year'] = df['Date'].dt.year
    annual_vol = df.groupby('Year')['Daily Return'].std() * np.sqrt(252)
    return annual_vol.rename(symbol)

# Caricamento dei dati
spy = pd.read_csv('Stocks/SPY.csv')
dia = pd.read_csv('Stocks/DIA.csv')
qqq = pd.read_csv('Stocks/QQQ.csv')

# Formattazione Datetime
spy['Date'] = pd.to_datetime(spy['Date'])
dia['Date'] = pd.to_datetime(dia['Date'])
qqq['Date'] = pd.to_datetime(qqq['Date'])

# Filtra i dati tra il 2014-01-01 e il 2024-01-01
start_date = '2014-01-01'
end_date = '2023-12-31'

spy = spy[(spy['Date'] >= start_date) & (spy['Date'] <= end_date)]
dia = dia[(dia['Date'] >= start_date) & (dia['Date'] <= end_date)]
qqq = qqq[(qqq['Date'] >= start_date) & (qqq['Date'] <= end_date)]

# Calcola la volatilità annuale per ciascun indice
spy_vol = annual_volatility(spy, 'SPY')
dia_vol = annual_volatility(dia, 'DIA')
qqq_vol = annual_volatility(qqq, 'QQQ')

# Unisce i dati in un unico DataFrame
volatility_df = pd.concat([spy_vol, dia_vol, qqq_vol], axis=1).reset_index()

# Creazione del grafico a barre con Plotly
fig = px.bar(volatility_df, x='Year', y=['SPY', 'DIA', 'QQQ'],
             barmode='group',
             labels={'value': 'Volatilità Annuale', 'variable': 'Indice'},
             title='Volatilità Annuale degli Indici Principali (2014-2023)')

# Titoli e etichette
fig.update_layout(
    xaxis_title='Anno',
    yaxis_title='Volatilità Annuale',
    xaxis=dict(
        tickmode='array',
        tickvals=volatility_df['Year'],
        ticktext=volatility_df['Year'].astype(str)
    )
)

# Mostra il grafico
fig.show()



**Osservazioni:**

- Volatilità significativamente aumentata nel 2020 per tutti gli indici.

- QQQ ha generalmente mostrato una maggiore volatilità rispetto a SPY e DIA.

### **Volume Medio Annuale degli Indici Principali (2014-2023)**

- SPY (S&P 500)

- DIA (Dow Jones Industrial Average)

- QQQ (NASDAQ-100)


In [4]:
# Funzione per calcolare il volume medio annuale
def annual_volume(df, symbol):
    df = df.copy()
    df['Year'] = df['Date'].dt.year
    annual_vol = df.groupby('Year')['Volume'].mean()
    return annual_vol.rename(symbol)

# Caricamento dei dati
spy = pd.read_csv('Stocks/SPY.csv')
dia = pd.read_csv('Stocks/DIA.csv')
qqq = pd.read_csv('Stocks/QQQ.csv')

# Formattazione Datetime
spy['Date'] = pd.to_datetime(spy['Date'])
dia['Date'] = pd.to_datetime(dia['Date'])
qqq['Date'] = pd.to_datetime(qqq['Date'])

# Filtraggio dei dati tra il 2014-01-01 e il 2024-01-01
start_date = '2014-01-01'
end_date = '2023-12-31'

spy = spy[(spy['Date'] >= start_date) & (spy['Date'] <= end_date)]
dia = dia[(dia['Date'] >= start_date) & (dia['Date'] <= end_date)]
qqq = qqq[(qqq['Date'] >= start_date) & (qqq['Date'] <= end_date)]

# Calcolo del volume medio annuale per ciascun indice
spy_volume = annual_volume(spy, 'SPY')
dia_volume = annual_volume(dia, 'DIA')
qqq_volume = annual_volume(qqq, 'QQQ')

# Unione i dati in un unico DataFrame per il volume
volume_df = pd.concat([spy_volume, dia_volume, qqq_volume], axis=1).reset_index()

# Creazione del grafico a barre con Plotly per il volume medio annuale
fig_volume = px.bar(volume_df, x='Year', y=['SPY', 'DIA', 'QQQ'],
             barmode='group',
             labels={'value': 'Volume Medio Annuale', 'variable': 'Indice'},
             title='Volume Medio Annuale degli Indici Principali (2014-2023)')

# Titoli e etichette
fig_volume.update_layout(
    xaxis_title='Anno',
    yaxis_title='Volume Medio Annuale',
    xaxis=dict(
        tickmode='array',
        tickvals=volume_df['Year'],
        ticktext=volume_df['Year'].astype(str)
    )
)

# Mostra il grafico
fig_volume.show()


**Osservazioni:**

- SPY ha il volume medio annuale più alto, riflettendo la sua popolarità e liquidità.

- QQQ mostra una tendenza crescente nel volume, specialmente dopo il 2017.

### **Performance Annuale dei Settori (Ultimi 5 Anni)**

- Tecnologia (XLK)

- Energia (KLE)

- Sanità (KLV)

- Industria (KLI)

- Beni di Consumo (XLP)


In [5]:
# Funzione per calcolare la performance annuale
def annual_performance(df, symbol):
    df = df.copy()
    df['Year'] = df['Date'].dt.year
    annual_performance = df.groupby('Year')['Close'].apply(lambda x: (x.iloc[-1] / x.iloc[0]) - 1)
    return annual_performance.rename(symbol)

# Caricamento dei dati di ciascun settore
xlk = pd.read_csv('Stocks/XLK.csv')  # Tecnologia
xle = pd.read_csv('Stocks/XLE.csv')  # Energia
xlv = pd.read_csv('Stocks/XLV.csv')  # Sanità
xli = pd.read_csv('Stocks/XLI.csv')  # Industria
xlp = pd.read_csv('Stocks/XLP.csv')  # Beni di Consumo

# Formattazione Datetime
xlk['Date'] = pd.to_datetime(xlk['Date'])
xle['Date'] = pd.to_datetime(xle['Date'])
xlv['Date'] = pd.to_datetime(xlv['Date'])
xli['Date'] = pd.to_datetime(xli['Date'])
xlp['Date'] = pd.to_datetime(xlp['Date'])

# Filtra i dati tra il 2019-01-01 e il 2024-01-01
start_date = '2019-01-01'
end_date = '2024-01-01'

xlk = xlk[(xlk['Date'] >= start_date) & (xlk['Date'] <= end_date)]
xle = xle[(xle['Date'] >= start_date) & (xle['Date'] <= end_date)]
xlv = xlv[(xlv['Date'] >= start_date) & (xlv['Date'] <= end_date)]
xli = xli[(xli['Date'] >= start_date) & (xli['Date'] <= end_date)]
xlp = xlp[(xlp['Date'] >= start_date) & (xlp['Date'] <= end_date)]

# Calcola la performance annuale per ciascun settore
xlk_perf = annual_performance(xlk, 'Tecnologia')
xle_perf = annual_performance(xle, 'Energia')
xlv_perf = annual_performance(xlv, 'Sanità')
xli_perf = annual_performance(xli, 'Industria')
xlp_perf = annual_performance(xlp, 'Beni di Consumo')

# Unione dei dati in un unico DataFrame
performance_df = pd.concat([xlk_perf, xle_perf, xlv_perf, xli_perf, xlp_perf], axis=1).reset_index()

# Trasforma il DataFrame in un formato lungo per Plotly
performance_long = performance_df.melt(id_vars='Year', value_vars=['Tecnologia', 'Energia', 'Sanità', 'Industria', 'Beni di Consumo'],
                                       var_name='Settore', value_name='Performance Annuale')

# Creazione del grafico a barre con Plotly
fig = px.bar(performance_long, x='Year', y='Performance Annuale', color='Settore', barmode='group',
             title='Performance Annuale dei Settori (Ultimi 5 Anni)',
             labels={'Performance Annuale': 'Performance Annuale', 'Year': 'Anno'})

# Titoli e etichette
fig.update_layout(
    xaxis_title='Anno',
    yaxis_title='Performance Annuale'
)

# Mostra il grafico
fig.show()


**Osservazioni:**

- Il settore della Tecnologia ha mostrato una forte performance durante gli ultimi 5 anni, tranne nel 2022.

- Il settore dell'Energia ha avuto una forte variabilità con cali nel 2020 e recuperi nel 2021 e 2022.

### **Matrice di Correlazione dei Rendimenti delle Azioni FAANG**

- META (Meta)

- AAPL (Apple)

- AMZN (Amazon)

- NFLX (Netflix)

- GOOG (Google)


In [6]:
# Funzione per calcolare i rendimenti giornalieri
def daily_returns(df):
    df = df.copy()
    df['Daily Return'] = df['Close'].pct_change()
    return df['Daily Return']

# Caricamento dei dati
meta = pd.read_csv('Stocks/META.csv')    # Meta Platforms (ex Facebook)
aapl = pd.read_csv('Stocks/AAPL.csv')    # Apple
amzn = pd.read_csv('Stocks/AMZN.csv')    # Amazon
nflx = pd.read_csv('Stocks/NFLX.csv')    # Netflix
goog = pd.read_csv('Stocks/GOOG.csv')    # Google

# Formattazione Datetime
meta['Date'] = pd.to_datetime(meta['Date'])
aapl['Date'] = pd.to_datetime(aapl['Date'])
amzn['Date'] = pd.to_datetime(amzn['Date'])
nflx['Date'] = pd.to_datetime(nflx['Date'])
goog['Date'] = pd.to_datetime(goog['Date'])

# Calcolo dei rendimenti giornalieri per ciascuna azione
meta_returns = daily_returns(meta).rename('META')
aapl_returns = daily_returns(aapl).rename('AAPL')
amzn_returns = daily_returns(amzn).rename('AMZN')
nflx_returns = daily_returns(nflx).rename('NFLX')
goog_returns = daily_returns(goog).rename('GOOG')

# Unione dei rendimenti in un unico DataFrame
returns_df = pd.concat([meta_returns, aapl_returns, amzn_returns, nflx_returns, goog_returns], axis=1).dropna()

# Calcolo della matrice di correlazione
correlation_matrix = returns_df.corr()

# Creazione del grafico heatmap con Plotly
fig = px.imshow(correlation_matrix, text_auto=True, aspect="auto",
                title='Matrice di Correlazione dei Rendimenti delle Azioni FAANG',
                labels=dict(color='Correlazione'))

# Titoli e etichette
fig.update_layout(
    xaxis_title='Azione',
    yaxis_title='Azione'
)

# Mostra il grafico
fig.show()


**Osservazioni:**

- La correlazione più alta è tra GOOG e AMZN (0.64).

- Correlazione moderata tra META e GOOG (0.63).

- AAPL mostra correlazioni moderate con AMZN (0.54) e GOOG (0.59).

- NFLX tende ad avere correlazioni più basse con le altre azioni.

### **Matrice di Correlazione tra Azioni e Materie Prime**

**Aziende Coinvolte**:

  - XOM (ExxonMobil): Petrolio e gas naturale

  - CVX (Chevron): Petrolio e gas naturale

  - NEM (Newmont Corporation): Oro e metalli preziosi

  - FCX (Freeport-McMoRan): Rame e metalli industriali

  - COP (ConocoPhillips): Petrolio e gas naturale

**Dataset Materie Prime**:

  - Oro, Argento e Rame: [Precious Metals Data](https://www.kaggle.com/datasets/guillemservera/precious-metals-data)

  - Petrolio e Gas naturale: [Fuels Futures Data](https://www.kaggle.com/datasets/guillemservera/fuels-futures-data)


In [7]:
# Funzione per calcolare i rendimenti giornalieri
def daily_returns(df):
    df = df.copy()
    df['Daily Return'] = df['Close'].pct_change()
    return df['Daily Return']

# Caricamento dei dati delle azioni
symbols_stocks = ['XOM', 'CVX', 'NEM', 'FCX', 'COP']
stocks_data = {symbol: pd.read_csv(f'Stocks/{symbol}.csv') for symbol in symbols_stocks}

# Carica i dati delle materie prime
symbols_commodities = ['GOLD', 'SILVER', 'COPPER', 'OIL', 'GAS']
commodities_data = {symbol: pd.read_csv(f'Stocks/{symbol}.csv') for symbol in symbols_commodities}

# Formattazione Datetime
for symbol in symbols_stocks + symbols_commodities:
    if symbol in stocks_data:
        stocks_data[symbol]['Date'] = pd.to_datetime(stocks_data[symbol]['Date'])
        stocks_data[symbol].set_index('Date', inplace=True)
    if symbol in commodities_data:
        commodities_data[symbol]['Date'] = pd.to_datetime(commodities_data[symbol]['Date'])
        commodities_data[symbol].set_index('Date', inplace=True)

# Filtro delle date tra il 2014-01-01 e il 2024-06-01
start_date = '2014-01-01'
end_date = '2024-06-01'

for symbol in symbols_stocks:
    stocks_data[symbol] = stocks_data[symbol].loc[start_date:end_date]

for symbol in symbols_commodities:
    commodities_data[symbol] = commodities_data[symbol].loc[start_date:end_date]

# Calcolo dei rendimenti giornalieri
stocks_returns = {symbol: daily_returns(stocks_data[symbol]).rename(symbol) for symbol in symbols_stocks}
commodities_returns = {symbol: daily_returns(commodities_data[symbol]).rename(symbol) for symbol in symbols_commodities}

# Combina tutti i rendimenti in un unico DataFrame
all_returns = pd.concat([stocks_returns[symbol] for symbol in symbols_stocks] +
                        [commodities_returns[symbol] for symbol in symbols_commodities], axis=1).dropna()

# Calcolo della matrice di correlazione
correlation_matrix = all_returns.corr()

# Filtraggio della matrice di correlazione per mostrare solo aziende contro materie prime
filtered_correlation_matrix = correlation_matrix.loc[symbols_commodities, symbols_stocks]

# Creazione del grafico heatmap con Plotly
fig = px.imshow(filtered_correlation_matrix, text_auto=True, aspect="auto",
                title='Matrice di Correlazione tra Azioni e Materie Prime',
                labels=dict(color='Correlazione'))

# Titoli e etichette
fig.update_layout(
    xaxis_title='Aziende',
    yaxis_title='Materie Prime'
)

# Mostra il grafico
fig.show()


**Osservazioni:**

- NEM e FCX mostrano una correlazione moderata con oro, argento e rame.

- XOM, CVX e COP mostrano una bassa correlazione con petrolio e gas naturale, dovuto alla natura diversificata delle operazioni di queste aziende.

### **Backtesting della Strategia di Trading basata sulle Medie Mobili**

- **Segnali di Acquisto**: Generati quando la media mobile a 50 giorni incrocia verso l'alto la media mobile a 200 giorni.

- **Segnali di Vendita**: Generati quando la media mobile a 50 giorni incrocia verso il basso la media mobile a 200 giorni.


In [8]:
# Caricamento dei dati
data = pd.read_csv('Stocks/SPY.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Parametri delle medie mobili
short_window = 50
long_window = 200

# Calcolo delle medie mobili
data['Short_MA'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
data['Long_MA'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

# Generazione dei segnali di trading
data['Signal'] = 0
data.loc[data.index[short_window:], 'Signal'] = np.where(data['Short_MA'][short_window:] > data['Long_MA'][short_window:], 1, 0)
data['Position'] = data['Signal'].diff()

# Esecuzione del backtesting
initial_capital = 100000.0
positions = data['Signal'] * 100
portfolio = pd.DataFrame(index=data.index)
portfolio['Holdings'] = positions * data['Close']
portfolio['Cash'] = initial_capital - (positions.diff() * data['Close']).fillna(0).cumsum()
portfolio['Total'] = portfolio['Cash'] + portfolio['Holdings']
portfolio['Returns'] = portfolio['Total'].pct_change()

# Plot dei risultati con Plotly
fig = go.Figure()

# Prezzo di chiusura
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Close'],
    mode='lines',
    name='Prezzo di Chiusura',
    line=dict(color='blue', width=2),
    opacity=0.5,
    hovertemplate='Data: %{x|%Y-%m-%d}<br>Prezzo di Chiusura: %{y:.2f}'
))

# Medie mobili
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Short_MA'],
    mode='lines',
    name='Media Mobile 50 giorni',
    line=dict(color='orange', width=2),
    opacity=0.75,
    hovertemplate='Data: %{x|%Y-%m-%d}<br>Media Mobile 50 giorni: %{y:.2f}'
))
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Long_MA'],
    mode='lines',
    name='Media Mobile 200 giorni',
    line=dict(color='green', width=2),
    opacity=0.75,
    hovertemplate='Data: %{x|%Y-%m-%d}<br>Media Mobile 200 giorni: %{y:.2f}'
))

# Segnali di acquisto e vendita
buy_signals = data[data['Position'] == 1]
sell_signals = data[data['Position'] == -1]

fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals['Short_MA'],
    mode='markers',
    name='Segnale di Acquisto',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    hovertemplate='Data: %{x|%Y-%m-%d}<br>Segnale di Acquisto: %{y:.2f}'
))
fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals['Short_MA'],
    mode='markers',
    name='Segnale di Vendita',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    hovertemplate='Data: %{x|%Y-%m-%d}<br>Segnale di Vendita: %{y:.2f}'
))

# Titolo e layout
fig.update_layout(
    title='Backtesting della Strategia di Trading basata sulle Medie Mobili',
    xaxis_title='Data',
    yaxis_title='Prezzo',
    legend_title='Legenda',
    hovermode='x'
)

# Mostra il grafico
fig.show()

# Performance del portafoglio
print(f"Rendimento totale: {portfolio['Total'].iloc[-1] / initial_capital - 1:.2%}")
print(f"Max Drawdown: {((portfolio['Total'].cummax() - portfolio['Total']) / portfolio['Total'].cummax()).max():.2%}")


Rendimento totale: 24.42%
Max Drawdown: 10.42%


## **Conclusioni**

In sintesi, le analisi condotte hanno evidenziato alcuni punti chiave:

- **Resilienza degli Indici**: I principali indici di mercato hanno mostrato una resilienza post-2020, con il settore tecnologico che ha registrato la crescita più significativa.

- **Volatilità Annuale**: La volatilità annuale ha avuto un picco nel 2020 a causa della pandemia di COVID-19, con il NASDAQ-100 che ha mostrato la maggiore volatilità.

- **Volume di Scambio**: Il volume medio di scambio dell'S&P 500 è stato costantemente il più alto, riflettendo la sua popolarità.

- **Performance dei Settori**: Tra i settori analizzati, il settore tecnologico ha mostrato forti performance tranne nel 2022, mentre il settore dell'energia ha avuto una performance variabile con forti recuperi nel 2021 e 2022.

- **Correlazioni**:

  - Correlazione moderata tra le azioni FAANG.
  
  - Correlazione moderata delle azioni NEM e FCX con argento, oro e rame. Correlazione Scarsa delle azioni XOM, CVX, COP con gas naturale e petrolio.

- **Backtesting della Strategia**: Ottenuto un profitto del 24.42% con la strategia di trading basata sulle medie mobili applicata  sull'azione SPY negli ultimi 10 anni.

Questi risultati forniscono una panoramica delle dinamiche del mercato e possono aiutare a informare decisioni di investimento future.
